# TP3

# <font color='blue'> Exercise</font>

Résolvons $Ax = b$
avec
$$
A = \begin{pmatrix}
   2 & -1 & 0 & 0 & \cdots & 0 & 0\\
   -1 & 2 & -1 & 0 &\cdots & 0 & 0 \\
   0 & -1 & 2 & -1 &\cdots & 0 & 0 \\
   0 & 0 & -1 & 2 &\cdots & 0 & 0 \\
   \vdots  & \vdots & \vdots & \vdots& \ddots & \vdots & \vdots  \\
   0 & 0 & 0 & 0 & \cdots & 2 & -1 \\
   0 & 0 & 0 & 0 & \cdots & -1 & 2
 \end{pmatrix} \in \mathbb{R}^{n \times n} = \mathscr{M}_n(\mathbb{R})
 \text{ and }
 b =
 \begin{pmatrix}
 1 \\
 1 \\
 \vdots \\
 1 \\
 1
 \end{pmatrix} \in \mathbb{R}^n
$$

Il est suffisant de minimiser la fonction :       
$$F : x \in \mathbb{R}^n \mapsto \frac{1}{2}\langle Ax,x \rangle - \langle b, x \rangle$$

1. Implémentez la matrice ```A``` et le vecteur ```b``` dans ```torch``` avec $n = 20$
2. Calculez le gradient d'un vecteur aléatoire ```x``` par rapport à ```F``` avec ```torch.autograd```
3. Vérifiez que le calcul du gradient est correct.
4. Implémentez-le en utilisant ```torch.optim.SGD```, un taux d'apprentissage égal à 0,1, un momentum égal à 0,9 et ```10**3``` itérations.

In [ ]:
#1. #TODO

In [ ]:
#2. #TODO

In [ ]:
#3. #TODO

In [ ]:
#4.

Nit = #TODO
optim = torch.optim.SGD(___, lr=___,momentum=__) #TODO

for k in range(Nit) :
  F = #TODO
         #Compute the gradient
  optim.step()
         #And ?


# Neural Networks



Les réseaux de neurones peuvent être construits à l'aide du package ``torch.nn``.

Maintenant que vous avez eu un aperçu de « autograd », « nn » dépend de
``autograd`` pour définir des modèles et les différencier.
Un ``nn.Module`` contient des couches et une méthode ``forward(input)`` qui
renvoie la « sortie ».

Une procédure d'entraînement typique pour un réseau de neurones est la suivante :

- Définir le réseau neuronal qui possède certains paramètres à apprendre (ou
  poids)
- Itérer sur un ensemble de données d'entrées
- Calculer la perte (dans quelle mesure la sortie est-elle correcte)
- Faire de la backpropagation dans les paramètres du réseau
- Mettre à jour les poids du réseau, généralement à l'aide d'une simple règle de mise à jour :
  ``poids = poids - taux d'apprentissage * gradient``

## Définir le réseau

Définissons un réseau :

In [ ]:
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

 <font color='blue'> **Question** : Décrire l'architecture du réseau </font>

 <font color='blue'> **Question** : Afficher les poids et biais de la première convolution </font>

Il suffit de définir la fonction ``forward`` et la fonction ``backward``
la fonction (où les gradients sont calculés) est automatiquement définie pour vous
en utilisant ``autograd``.
Vous pouvez utiliser n'importe laquelle des opérations Tensor dans la fonction « forward ».

Les paramètres apprenables d'un modèle sont renvoyés par ``net.parameters()``



In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

**Attention !**
Quand on parle du nombre de paramètres d'un réseau, on parle du nombre de nombres réels dans les matrices, biais et noyaux impliqués dans le réseau (comme dans le cours).

 <font color='blue'> **Question** : Combien de paramètres a ce réseau ? </font>

Essayons d'envoyer une image aléatoire en entrée.


 <font color='blue'> **Question** : Testez le réseau sur une entrée aléatoire qui est une image en niveaux de gris (un canal) de taille $32 \times 32$.</font>

In [ ]:
input = #TODO
out = net(input)
print(out)

##  <font color='blue'> Exercice </font>

1. Nous aimerions définir un réseau qui classe les images de taille $(3,32,32)$.
Construire une classe Net() qui applique :
- une convolution avec $6$ canaux en sortie et un noyau de taille $5 \times 5$, suivi d'un ReLu
- un max pooling de taille $2 \times 2$
- une convolution avec $16$ de canaux en sortie et un noyau de taille $3 \times 3$, suivi d'un ReLu
- un  max pooling de taille $2 \times 2$
- une couche linéaire avec une taille de sortie de $120$, suivie d'un ReLu
- une couche linéaire avec une taille de sortie de $84$, suivie d'un ReLu
- une couche linéaire avec une taille de sortie de $10$

Vérifiez que cela fonctionne avec une entrée aléatoire.

2. Dessinez le réseau comme cela est fait dans les dernières slides du cours.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #TODO

    def forward(self, x):
      #TODO


net = Net()

In [ ]:
x = #TODO
net(x)

# Retour sur le TP1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_moons, make_circles, make_classification, make_blobs, make_gaussian_quantiles
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X, t = make_gaussian_quantiles(n_features=2, n_classes=3, n_samples=500)

X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=.4, random_state=12)
# Number of points in each set:
N_train = X_train.shape[0]
N_test = X_test.shape[0]

figure = plt.figure(figsize=(10, 10))
plt.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
plt.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k');
plt.show()

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
t_train = torch.tensor(t_train, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
t_test = torch.tensor(t_test, dtype=torch.int64)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

d = 5
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2,d)
        self.fc2 = nn.Linear(d, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()
net2 = Net()


<font color='blue'> Question : Reconnaissez-vous le réseau ci-dessus ?</font>

In [ ]:
net.fc1.bias.requires_grad = False
net.fc1.weight.requires_grad = False


<font color='blue'> Question : Qu'est-ce qui est fait ci-dessus ?</font>

In [ ]:


W1 =torch.randn(d,2)
b1 = torch.randn(d)
net.fc2.bias = torch.nn.Parameter(b1)
net.fc2.weight = torch.nn.Parameter(W1)
net2.fc2.bias = torch.nn.Parameter(b1)
net2.fc2.weight = torch.nn.Parameter(W1)


W2 =torch.randn(3,d)
b2 = torch.randn(3)
net.fc2.bias = torch.nn.Parameter(b2)
net.fc2.weight = torch.nn.Parameter(W2)
net2.fc2.bias = torch.nn.Parameter(b2)
net2.fc2.weight = torch.nn.Parameter(W2)

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.75)
optimizer2 = optim.SGD(net2.parameters(), lr=0.75)

In [ ]:

for epoch in range(10**3):  # loop over the dataset multiple times
    # zero the parameter gradients
    optimizer.zero_grad()
    optimizer2.zero_grad()
    # forward + backward + optimize
    output = net(X_train)
    loss = criterion(output, t_train)
    loss.backward()
    optimizer.step()
    # forward + backward + optimize
    output2 = net2(X_train)
    loss2 = criterion(output2, t_train)
    loss2.backward()
    optimizer2.step()

print('Finished Training')


In [ ]:
#visualize results:

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = 0.02
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
X_grid = np.hstack((xx.ravel(), yy.ravel()))

N_grid = xx.ravel().shape[0]
X_grid = np.c_[xx.ravel(), yy.ravel()]

feature_transform = lambda x : (net(torch.tensor(x, dtype=torch.float32).unsqueeze(0)).detach().numpy())
feature_transform2 = lambda x : (net2(torch.tensor(x, dtype=torch.float32).unsqueeze(0)).detach().numpy())


Phi_grid = feature_transform(X_grid)
Phi_grid2 = feature_transform2(X_grid)

Z =np.argmax(Phi_grid,axis=2)
Z = Z.reshape(xx.shape)

figure = plt.figure(figsize=(16, 8))
ax = plt.subplot(1,2,1)
ax.set_title("Input data")
ax.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
ax.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')
ax = plt.subplot(1,2,2)
cmap = ListedColormap(['b','y','r','m','g','c'])
plt.contourf(xx,yy,Z,  cmap = cmap, alpha=.8)
ax.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
ax.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')

Z2 =np.argmax(Phi_grid2,axis=2)
Z2 = Z2.reshape(xx.shape)
figure = plt.figure(figsize=(16, 8))
ax = plt.subplot(1,2,1)
ax.set_title("Input data")
ax.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
ax.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')
ax = plt.subplot(1,2,2)
cmap = ListedColormap(['b','y','r','m','g','c'])
plt.contourf(xx,yy,Z2,  cmap = cmap, alpha=.8)
ax.scatter(X_train[:, 0], X_train[:, 1], marker='o', c=t_train, s=50, edgecolor='k')
ax.scatter(X_test[:, 0], X_test[:, 1], marker='P', c=t_test, s=50, edgecolor='k')

# Test image

In [ ]:
!git clone 'https://github.com/emilePi/denis'

In [ ]:
import io
import requests
from PIL import Image
from torchvision.transforms.functional import to_tensor
import pylab as plt
import numpy as np
img = to_tensor(Image.open('./denis/denis.jpeg'))
npimg = img.numpy()
plt.imshow(np.transpose(npimg, (1, 2, 0)),cmap=plt.cm.gray)
plt.show()


In [ ]:
import torch
from torchvision.transforms.functional import to_tensor, to_pil_image
from IPython.display import display
from PIL import Image

im_pil = Image.open('./denis/denis.jpeg')
display(im_pil)